[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/g-emarco/notebook_host/blob/main/pandasai_v3_showcase.ipynb)

# PandasAI v3: The Complete Showcase

This notebook provides a comprehensive overview of the capabilities of PandasAI v3, covering a wide range of functions from basic setup to advanced features like agents, custom skills, and the semantic layer.

## Section 1: Setup and Basic Chat

In [ ]:
import pandas as pd
import os
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM
# Настройка API ключа (замените на ваш реальный ключ)
# В реальном проекте рекомендуется использовать переменные окружения
api_key = os.getenv("OPENAI_API_KEY", "your-api-key")
llm = LiteLLM(model="gpt-5-mini", api_key=api_key)
# Настройка конфигурации PandasAI через глобальный config
pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})



In [ ]:
# 1.2 - Create a sample DataFrame and use df.to_csv
data = {
    'country': ['United States', 'United Kingdom', 'France', 'Germany', 'Italy', 'Spain', 'Canada', 'Australia', 'Japan', 'China'],
    'gdp': [21.4, 2.8, 2.7, 3.8, 2.0, 1.4, 1.7, 1.4, 5.1, 14.3],
    'happiness_index': [7.0, 7.1, 6.8, 7.2, 6.5, 6.6, 7.3, 7.2, 6.9, 5.1]
}
df = pd.DataFrame(data)
df.to_csv('country_data.csv', index=False)



In [ ]:
# 1.3 - pai.read_csv
countries_df = pai.read_csv('country_data.csv')



In [ ]:
# 1.4 - df.chat (string output)
print(countries_df.chat('Which country has the highest GDP?'))

# 1.5 - df.chat (dataframe output)
print(countries_df.chat('What are the top 3 countries by happiness index?'))


In [3]:
# 1.6 - df.chat (chart output) and chart_response.save
chart = countries_df.chat(
    'Plot a bar chart of the GDP of the top 5 countries by happiness index.')
chart.save('gdp_chart.png')


## Section 2: The PandasAI Agent

In [4]:
from pandasai import Agent

# The v3 Agent maintains conversational context automatically.
agent = Agent(countries_df)  # Use verbose=True to see the generated code
print(agent.chat('Which are the bottom 3 countries by GDP?'))
# No follow_up method needed
print(agent.chat('Of these, which one has the highest happiness index?'))


     country  gdp  happiness_index
0  Australia  1.4              7.2
1      Spain  1.4              6.6
2     Canada  1.7              7.3
Of the bottom 3 by GDP, the country with the highest happiness index is Canada with a happiness index of 7.3.


## Section 3: LLMs and Database Connectors

In [6]:
from pandasai.data_loader.duck_db_connection_manager import DuckDBConnectionManager

# Create an in-memory DuckDB database
db_manager = DuckDBConnectionManager()
db_manager.register('countries', countries_df)

db_agent = Agent(countries_df)
print(db_agent.chat('Which country has the highest GDP?'))


The country with the highest GDP is United States with GDP of $21.40.


## Section 4: Vector Stores and Agent Training

In [8]:
from pandasai_lancedb import LanceDB
from pandasai_chromadb import ChromaDB
from pandasai_pinecone import Pinecone
from pandasai_qdrant import Qdrant

# Using LanceDB
lancedb_store = LanceDB(persist_path="./lancedb")
lance_agent = Agent(countries_df, vectorstore=lancedb_store)
lance_agent.train(queries=["What is the happiest country?"], codes=[
                  "print(df.sort_values(by='happiness_index', ascending=False).iloc[0]['country'])"])
print(lance_agent.chat("What is the happiest country?"))

# Conceptual examples for other vector stores
# pinecone_store = Pinecone(api_key="YOUR_PINECONE_API_KEY")
# qdrant_store = Qdrant(path="./qdrant_local")
# chromadb_store = ChromaDB(persist_path="./chromadb")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The happiest country is Canada with a happiness index of 7.3.
The happiest country is Canada with a happiness index of 7.3.
The happiest country is Canada with a happiness index of 7.3.
The happiest country is Canada with a happiness index of 7.3.


InvalidOutputValueMismatch: Result must be in the format of dictionary of type and value like `result = {"type": ..., "value": ... }`